In [22]:
import bioalerts
from bioalerts import LoadMolecules, Alerts, FPCalculator
import sys

In [23]:
import numpy as np 
import matplotlib.pyplot as plt

from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem import PandasTools 

import pandas as pd 

from sklearn.neighbors import KNeighborsRegressor #knn
from sklearn.ensemble import RandomForestRegressor #RF
from sklearn.svm import SVR #SVR

from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedKFold

In [24]:
smiles_path = "./tutorial/datasets/lipophilicity.smi"
bio_path = './tutorial/datasets/lipophilicity.bio.txt'

wdir =  'C:/jupyter_devel/kist-europe/QSAR/AOP_data/moleculenet_dataset/'
csv = 'Lipophilicity_check-maccs-remcols-stdval.csv'
result_wdir = 'C:/jupyter_devel/kist-europe/QSAR/AOP_data/moleculenet_dataset/'

In [25]:
df = pd.read_csv(wdir+csv)

In [26]:
y = df['Standard Value']
x = df.drop(['Molecule','Standard Value'],axis=1)

## A. Data Visualization

In [27]:
x_ = range(len(x))
plt.scatter(x_,y)
#plt.ylim(0,50000)

#### PIC transformation Formula ####
# PIC = 9 - np.log10(actual activity)
# actual activity = np,power(10,9-PIC)

## B. calculate significant substructure feature

In [28]:
molecules = bioalerts.LoadMolecules.LoadMolecules(smiles_path,name_field=None)
molecules.ReadMolecules()

Format of the structures file = SMILES
All molecules in the input file were processed correctly


In [29]:
cox_bio = np.genfromtxt(bio_path,skip_header=0) 
cox_bio.shape
arr = np.arange(0,len(cox_bio))
mask = np.ones(arr.shape,dtype=bool)
mask[molecules.molserr]=0
cox_bio = cox_bio[mask]
print len(cox_bio)
print len(molecules.mols)

4200
4200


In [30]:
dataset_info = bioalerts.LoadMolecules.GetDataSetInfo(name_field=None)
dataset_info.extract_substructure_information(radii=[2,3,4,5,6],mols=molecules.mols)

In [31]:
Alerts_continuous_high_activity = bioalerts.Alerts.CalculatePvaluesContinuous(radii_ext=[2,3,4,5,6])

In [32]:
def calculate_bioactivity_threshold(bio,threshold):
    # a pIC50 is smaller, higher activity 
    
    sorted_bio = sorted(bio)
    threshold_bio_value = sorted_bio[int(len(sorted_bio)*threshold)]
    
    print("activity ratio is {}".format(1-threshold))
    print("threshold value is {}".format(threshold_bio_value))
    print("actual bioactivity value is {}".format(np.power(10,9 - threshold_bio_value)))
    return threshold_bio_value

In [33]:
threshold_= 0.3
threshold_bio_value = calculate_bioactivity_threshold(cox_bio,threshold_)

threshold_nb_substructures = 5
threshold_ratio = 0.2
threshold_high_act_nb_substructures = 0 #5
threshold_high_act_ratio = 0.0 #0.5                                                                  

# maybe no necessary to change
threshold_pvalue = 0.05 

activity ratio is 0.7
threshold value is 1.64
actual bioactivity value is 22908676.5277


In [21]:
significant_substructure = {}
significant_substructure = Alerts_continuous_high_activity.get_significant_substructure_with_high_bioactivity(
mols = molecules.mols,
     substructure_dictionary = dataset_info.substructure_dictionary,
     bioactivities = cox_bio,
     mols_ids = molecules.mols_ids[:],
     threshold_nb_substructures = threshold_nb_substructures,
     threshold_pvalue = threshold_pvalue,
     threshold_ratio=threshold_ratio,
     threshold_high_act_nb_substructures=threshold_high_act_nb_substructures,
     threshold_high_act_ratio = threshold_high_act_ratio,                                                                        
     threshold_bioactivity=threshold_bio_value)

#threshold_high_act_nb_substructure = N of molecules involved significant substructure with high activity  
#threshold_high_act_ratio = N of molecules involved significant substructure with high activity / N of molecules involved significant substructure
#threshold_bioactivity = Top 40% by total bioactivity

#return value is significant_substructure dictionary with high activity

KeyboardInterrupt: 

In [ ]:
mol_id_set = set()
for k,v in significant_substructure.items():
    for mol_id in v: 
        mol_id_set.add(mol_id)

In [ ]:
print('number of total substructure : {}'.format(len(dataset_info.substructure_dictionary)))
print('number of extracted significant substructure : {}'.format(len(significant_substructure)))
print('{} out of {} have the label which represent remaining significant substructure\n'.format(len(mol_id_set),len(cox_bio)))

print ('THRESHOLD - \n\tnumber of substructures : {}\n\tp value : {}\n\tratio : {}\n\tnumber of high activity threshold : {}\n\thigh activity ratio : {}\n\thigh bioactivity threshold : {}'
       .format(threshold_nb_substructures,threshold_pvalue,threshold_ratio,threshold_high_act_nb_substructures,threshold_high_act_ratio,threshold_ ))

In [ ]:
# run the code when you repair hyperparameter
if len(significant_substructure) is not 0 :
    significant_substructure.clear()
if len(mol_id_set) is not 0 :
    mol_id_set.clear()

## C. insert feature

In [ ]:
y_pic50 = 9 - np.log10(y)

In [ ]:
#insert feature into only train dataset
for iter_ in x.index:
    for i in range(8): 
        if iter_ in mol_id_set : 
            x.loc[iter_,'significant_feature_{}'.format(i)] = 1.0
        else :
            x.loc[iter_,'significant_feature_{}'.format(i)] = 0.0

## D. model train

In [ ]:
test_size = 0.1 
x_train, x_test, y_train, y_test = train_test_split(x,y_pic50,test_size = test_size, shuffle = True)

In [7]:
test_size = 0.1 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = test_size, shuffle = True)

In [8]:
x_test_idx = []
for row in x_test.index:
    x_test_idx.append(row)

### D.1.1 KNN hyperparameter search 

In [ ]:
from sklearn.model_selection import GridSearchCV 

neigh = KNeighborsRegressor()
param_grid = {'n_neighbors':[4,5,6,7,8],
              'algorithm':['auto','ball_tree','brute'],
              'metric':['jaccard','matching','dice'],
             'weights':['uniform','distance']} 
KNN_search = GridSearchCV(neigh,param_grid,cv=5)
KNN_search.fit(x_train,y_train)
print("Bset Parameters set found on GridSearch CV :\n")
print(KNN_search.best_params_)

In [ ]:
print("Grid scores on GridSearchCV :\n")

means = KNN_search.cv_results_['mean_test_score']
stds = KNN_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, KNN_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
knn_search_pred = KNN_search.predict(x_test)

# test rseult 
print("\ntest RMSE : {}".format(round(np.sqrt(mean_squared_error(y_test,knn_search_pred)),4)) )

In [ ]:
KNN_search.best_params_

### D.1.2 KNN model train

In [9]:
#fold setting
kf = RepeatedKFold(n_splits = 5,n_repeats=10)
kf.get_n_splits(x_train)
#kf
#>>> KFold(n_splits=5, random_state=None, shuffle=False)

#for numbering k folds
k=0
cv_rmse = []
cv_rmse_mean = []
tmp = []

#DataFrame generate
y_pic_pred_result = pd.DataFrame()
y_pic_pred_result['idx'] = pd.Series([i for i in range(len(y))])


##################    using hyperparameter tuning model    ##################    
#model generate
neigh = KNeighborsRegressor(n_neighbors = 5, metric='matching',weights='distance',algorithm='brute')
#KNN without sig, for androgen {'n_neighbors': 5, 'metric': 'matching', 'weights': 'distance', 'algorithm': 'brute'}
##################    using hyperparameter tuning model    ##################


#cross-val training
for train_idx, test_idx in kf.split(x_train):
    #print("iteration num : {}".format(k))
    #print("train : ", train_idx, "\ntest : ", test_idx)
    x_cv_train , x_cv_test = x_train.iloc[train_idx], x_train.iloc[test_idx]
    y_cv_train , y_cv_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    neigh.fit(x_cv_train,y_cv_train)
    
    y_cv_testidx = test_idx
    y_cv_pred = neigh.predict(x_cv_test)
    cv_rmse.append(round(np.sqrt(mean_squared_error(y_cv_test,y_cv_pred)),4))
    cv_rmse_mean.append(round(np.sqrt(mean_squared_error(y_cv_test,y_cv_pred)),4))
    
    y_pic_pred_result['y_cv_testidx_{}'.format(k)] = pd.Series()
    y_pic_pred_result['y_cv_pred_{}'.format(k)] = pd.Series()
    
    for j in range(len(y_cv_testidx)):
        y_pic_pred_result.loc[y_cv_testidx[j],'y_cv_testidx_{}'.format(k)] = y_cv_testidx[j]
        y_pic_pred_result.loc[y_cv_testidx[j],'y_cv_pred_{}'.format(k)] = y_cv_pred[j]            
    
    #y_pic_pred_result['y_cv_testidx_{}'.format(i)] = pd.Series(y_cv_testidx)
    #y_pic_pred_result['y_cv_pred_{}'.format(i)] = pd.Series(y_cv_pred)
    k += 1 
    
    if k % 5 == 0 : 
        tmp.append(np.mean(cv_rmse_mean))
        print(cv_rmse_mean) # per fold rmse value in one cv 
        print("mean : ",np.mean(cv_rmse_mean)) # RMSE mean value per iteration of cv  
        del cv_rmse_mean[:]
        

print("total fold mean : ", round(np.mean(cv_rmse),4)) # mean of 50 cv rmse (1회 iter돌 때의 모든 rmse의 평균 (50회의 평균)) 
print("total cv mean : ",round(np.mean(tmp),4)) # mean of cv's mean of 1 iter (각 cv rmse 평균의 평균) # we need var of this value 

#model retrain with all train data  
neigh.fit(x_train,y_train) # > model train
y_pic_test_pred = neigh.predict(x_test) # > model predidction

# 
y_pic_test_pred_df = pd.DataFrame(columns=['y_pic_test_idx','y_pic_test_pred'])
for j in range(len(x_test_idx)):
        y_pic_test_pred_df.loc[x_test_idx[j],'y_pic_test_idx'] = x_test_idx[j]
        y_pic_test_pred_df.loc[x_test_idx[j],'y_pic_test_pred'] = y_pic_test_pred[j]
#y_pic_test_pred_df = pd.DataFrame(y_pic_test_pred,columns=['y_pic_test_pred'])
y_pic_pred_result = pd.concat([y_pic_pred_result, y_pic_test_pred_df], axis=1, ignore_index=False)

#save csv 
#y_pic_pred_result.to_csv(result_wdir+'5-alpha-reductase_dltnan_y_pic_pred_result+with_significant_feature.csv',index=True)


C:\Users\chon0\Anaconda3\envs\py27\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


[0.9723, 0.9463, 0.9381, 0.8685, 0.9543]
('mean : ', 0.9359)
[0.9797, 0.9566, 0.9256, 0.9378, 0.8727]
('mean : ', 0.9344800000000001)
[0.9255, 0.9039, 0.9409, 0.9542, 0.9625]
('mean : ', 0.9374)
[0.9204, 0.8948, 0.932, 0.9962, 0.9092]
('mean : ', 0.9305199999999999)
[0.9711, 0.9166, 0.918, 0.9233, 0.9498]
('mean : ', 0.9357599999999999)
[0.9017, 0.9779, 0.9422, 0.8964, 0.9684]
('mean : ', 0.93732)
[0.9317, 0.9586, 0.943, 0.9492, 0.9256]
('mean : ', 0.94162)
[0.8517, 0.9282, 1.0403, 0.9242, 0.9317]
('mean : ', 0.9352199999999999)
[0.9496, 0.9447, 0.8826, 0.9329, 0.9493]
('mean : ', 0.9318200000000001)
[0.9846, 0.9621, 0.8595, 0.9168, 0.9706]
('mean : ', 0.93872)
('total fold mean : ', 0.9359)
('total cv mean : ', 0.9359)


### D.1.3 KNN model results

In [10]:
#RESULT
print("train data size : {} ({}%)\n".format(len(x_train),round((len(x_train)/len(x)),2)), "test data size : {} ({}%)\n".format(len(x_test),round((len(x_test)/len(x)),2)))
print("each CV RMSE average : \n{} \nVARIANCE of RMSE of every folds : {}\n".format(tmp,round(np.var(cv_rmse),4)))

print("total 50 folds RMSE mean : ", round(np.mean(cv_rmse),4)) # mean of 50 cv rmse (1회 iter돌 때의 모든 rmse의 평균 (50회의 평균)) 
print("total 10 CVs RMSE mean : ",round(np.mean(tmp),4)) # mean of cv's mean of 1 iter (각 cv rmse 평균의 평균) # we need var of this value 

# test rseult 
print("\ntest RMSE : {}".format(round(np.sqrt(mean_squared_error(y_test,y_pic_test_pred)),4)) )

('train data size : 3780 (0.0%)\n', 'test data size : 420 (0.0%)\n')
each CV RMSE average : 
[0.9359, 0.9344800000000001, 0.9374, 0.9305199999999999, 0.9357599999999999, 0.93732, 0.94162, 0.9352199999999999, 0.9318200000000001, 0.93872] 
VARIANCE of RMSE of every folds : 0.0012

('total 50 folds RMSE mean : ', 0.9359)
('total 10 CVs RMSE mean : ', 0.9359)

test RMSE : 0.9046


### D.2.1 RF hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
RF = RandomForestRegressor()
RF_param_grid = {'n_estimators':[40,60,80,100],
              'max_depth':[30,40,50,None],
                'min_samples_split':[2,3,4,5,6]} 
RF_search = GridSearchCV(RF,RF_param_grid,cv=5)
RF_search.fit(x_train,y_train)
print("Bset Parameters set found on GridSearch CV :\n")
print(RF_search.best_params_)

In [ ]:
print("Grid scores on GridSearchCV :\n")

means = RF_search.cv_results_['mean_test_score']
stds = RF_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, RF_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

### D.2.2 RF model train

In [ ]:
test_size = 0.1 
x_train, x_test, y_train, y_test = train_test_split(x,y_pic50,test_size = test_size, shuffle = True)

In [11]:
test_size = 0.1 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = test_size, shuffle = True)

In [12]:
x_test_idx = []
for row in x_test.index:
    x_test_idx.append(row)

In [13]:
#fold setting
kf = RepeatedKFold(n_splits = 5,n_repeats=10)
kf.get_n_splits(x_train)
#kf
#>>> KFold(n_splits=5, random_state=None, shuffle=False)

#for numbering k folds
k=0
cv_rmse = []
cv_rmse_mean = []
tmp = []

#DataFrame generate
y_pic_pred_result = pd.DataFrame()
y_pic_pred_result['idx'] = pd.Series([i for i in range(len(y))])

#model generate
##################    using hyperparameter tuning model    ##################    
RF = RandomForestRegressor(n_estimators= 60,min_samples_split=4,max_depth=None)

#RF without sig, for androgen {'min_samples_split': 5, 'n_estimators': 40, 'max_depth': 40}
#RF with sig, for androgen 

#RF without sig, for aromatase {'min_samples_split': 3, 'n_estimators': 60, 'max_depth': 30}
#RF with sig , for aromatase {'min_samples_split': 2, 'n_estimators': 40, 'max_depth': None}
#
##################    using hyperparameter tuning model    ##################

#cross-val training
for train_idx, test_idx in kf.split(x_train):
    #print("iteration num : {}".format(k))
    #print("train : ", train_idx, "\ntest : ", test_idx)
    x_cv_train , x_cv_test = x_train.iloc[train_idx], x_train.iloc[test_idx]
    y_cv_train , y_cv_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    RF.fit(x_cv_train,y_cv_train)
    
    y_cv_testidx = test_idx
    y_cv_pred = RF.predict(x_cv_test)
    cv_rmse.append(round(np.sqrt(mean_squared_error(y_cv_test,y_cv_pred)),4))
    cv_rmse_mean.append(round(np.sqrt(mean_squared_error(y_cv_test,y_cv_pred)),4))
    
    y_pic_pred_result['y_cv_testidx_{}'.format(k)] = pd.Series()
    y_pic_pred_result['y_cv_pred_{}'.format(k)] = pd.Series()
    
    for j in range(len(y_cv_testidx)):
        y_pic_pred_result.loc[y_cv_testidx[j],'y_cv_testidx_{}'.format(k)] = y_cv_testidx[j]
        y_pic_pred_result.loc[y_cv_testidx[j],'y_cv_pred_{}'.format(k)] = y_cv_pred[j]
        
    
    
    #y_pic_pred_result['y_cv_testidx_{}'.format(i)] = pd.Series(y_cv_testidx)
    #y_pic_pred_result['y_cv_pred_{}'.format(i)] = pd.Series(y_cv_pred)
    k += 1 
    
    if k % 5 == 0 : 
        tmp.append(np.mean(cv_rmse_mean))
        print(cv_rmse_mean) # per fold rmse value in one cv 
        print("mean : ",np.mean(cv_rmse_mean)) # RMSE mean value per iteration of cv  
        del cv_rmse_mean[:]
        

print("total fold mean : ", round(np.mean(cv_rmse),4)) # mean of 50 cv rmse (1회 iter돌 때의 모든 rmse의 평균 (50회의 평균)) 
print("total cv mean : ",round(np.mean(tmp),4)) # mean of cv's mean of 1 iter (각 cv rmse 평균의 평균) # we need var of this value 

#model retrain with all train data  
RF.fit(x_train,y_train) # > model train
y_pic_test_pred = RF.predict(x_test) # > model predidction

# 
y_pic_test_pred_df = pd.DataFrame(columns=['y_pic_test_idx','y_pic_test_pred'])
for j in range(len(x_test_idx)):
        y_pic_test_pred_df.loc[x_test_idx[j],'y_pic_test_idx'] = x_test_idx[j]
        y_pic_test_pred_df.loc[x_test_idx[j],'y_pic_test_pred'] = y_pic_test_pred[j]
#y_pic_test_pred_df = pd.DataFrame(y_pic_test_pred,columns=['y_pic_test_pred'])
y_pic_pred_result = pd.concat([y_pic_pred_result, y_pic_test_pred_df], axis=1, ignore_index=False)

#save csv 
#y_pic_pred_result.to_csv(result_wdir+'5-alpha-reductase_dltnan_y_pic_pred_result+with_significant_feature.csv',index=True)


[0.8366, 0.8465, 0.801, 0.8161, 0.8158]
('mean : ', 0.8232000000000002)
[0.8385, 0.8475, 0.8169, 0.816, 0.7719]
('mean : ', 0.81816)
[0.8197, 0.7864, 0.8245, 0.7998, 0.8448]
('mean : ', 0.81504)
[0.8516, 0.7763, 0.8135, 0.8388, 0.8345]
('mean : ', 0.82294)
[0.7883, 0.8185, 0.834, 0.8971, 0.7857]
('mean : ', 0.8247199999999999)
[0.8346, 0.8069, 0.7929, 0.845, 0.8259]
('mean : ', 0.8210599999999999)
[0.8247, 0.8125, 0.814, 0.8552, 0.8478]
('mean : ', 0.83084)
[0.8455, 0.864, 0.7924, 0.8077, 0.7961]
('mean : ', 0.8211400000000001)
[0.8062, 0.8002, 0.8274, 0.8002, 0.8626]
('mean : ', 0.81932)
[0.8391, 0.8141, 0.7787, 0.824, 0.8437]
('mean : ', 0.81992)
('total fold mean : ', 0.8216)
('total cv mean : ', 0.8216)


### D.2.3 RF model results

In [14]:
#RESULT
print("train data size : {} ({}%)\n".format(len(x_train),round((len(x_train)/len(x)),2)), "test data size : {} ({}%)\n".format(len(x_test),round((len(x_test)/len(x)),2)))
print("each CV RMSE average : \n{} \nVARIANCE of RMSE of every folds : {}\n".format(tmp,round(np.var(cv_rmse),4)))

print("total 50 folds RMSE mean : ", round(np.mean(cv_rmse),4)) # mean of 50 cv rmse (1회 iter돌 때의 모든 rmse의 평균 (50회의 평균)) 
print("total 10 CVs RMSE mean : ",round(np.mean(tmp),4)) # mean of cv's mean of 1 iter (각 cv rmse 평균의 평균) # we need var of this value 

# test rseult 
print("\ntest RMSE : {}".format(round(np.sqrt(mean_squared_error(y_test,y_pic_test_pred)),4)) )

('train data size : 3780 (0.0%)\n', 'test data size : 420 (0.0%)\n')
each CV RMSE average : 
[0.8232000000000002, 0.81816, 0.81504, 0.82294, 0.8247199999999999, 0.8210599999999999, 0.83084, 0.8211400000000001, 0.81932, 0.81992] 
VARIANCE of RMSE of every folds : 0.0006

('total 50 folds RMSE mean : ', 0.8216)
('total 10 CVs RMSE mean : ', 0.8216)

test RMSE : 0.8381


### D.3.1 SVR hyperparameter tuning 

In [ ]:
from sklearn.model_selection import GridSearchCV
SVR = SVR()
SVR_param_grid = {'kernel':['rbf','linear','poly']} 
SVR_search = GridSearchCV(SVR,SVR_param_grid,cv=5)
SVR_search.fit(x_train,y_train)
print("Bset Parameters set found on GridSearch CV :\n")
print(SVR_search.best_params_)

In [ ]:
print("Grid scores on GridSearchCV :\n")

means = SVR_search.cv_results_['mean_test_score']
stds = SVR_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, SVR_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

### D.3.2 SVR model train

In [ ]:
#fold setting
kf = RepeatedKFold(n_splits = 5,n_repeats=10)
kf.get_n_splits(x_train)
#kf
#>>> KFold(n_splits=5, random_state=None, shuffle=False)

#for numbering k folds
k=0
cv_rmse = []
cv_rmse_mean = []
tmp = []

#DataFrame generate
y_pic_pred_result = pd.DataFrame()
y_pic_pred_result['idx'] = pd.Series([i for i in range(len(y))])

#model generate
##################    using hyperparameter tuning model    ##################    
SVR = SVR(kernel='rbf')
##################    using hyperparameter tuning model    ##################

#cross-val training
for train_idx, test_idx in kf.split(x_train):
    print("iteration num : {}".format(k))
    #print("train : ", train_idx, "\ntest : ", test_idx)
    x_cv_train , x_cv_test = x_train.iloc[train_idx], x_train.iloc[test_idx]
    y_cv_train , y_cv_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    SVR.fit(x_cv_train,y_cv_train)
    
    y_cv_testidx = test_idx
    y_cv_pred = SVR.predict(x_cv_test)
    cv_rmse.append(round(np.sqrt(mean_squared_error(y_cv_test,y_cv_pred)),4))
    cv_rmse_mean.append(round(np.sqrt(mean_squared_error(y_cv_test,y_cv_pred)),4))
    
    y_pic_pred_result['y_cv_testidx_{}'.format(k)] = pd.Series()
    y_pic_pred_result['y_cv_pred_{}'.format(k)] = pd.Series()
    
    for j in range(len(y_cv_testidx)):
        y_pic_pred_result.loc[y_cv_testidx[j],'y_cv_testidx_{}'.format(k)] = y_cv_testidx[j]
        y_pic_pred_result.loc[y_cv_testidx[j],'y_cv_pred_{}'.format(k)] = y_cv_pred[j]
        
    
    
    #y_pic_pred_result['y_cv_testidx_{}'.format(i)] = pd.Series(y_cv_testidx)
    #y_pic_pred_result['y_cv_pred_{}'.format(i)] = pd.Series(y_cv_pred)
    k += 1 
    
    if k % 5 == 0 : 
        tmp.append(np.mean(cv_rmse_mean))
        print(cv_rmse_mean) # per fold rmse value in one cv 
        print("mean : ",np.mean(cv_rmse_mean)) # RMSE mean value per iteration of cv  
        del cv_rmse_mean[:]
        

print("total fold mean : ", round(np.mean(cv_rmse),4)) # mean of 50 cv rmse (1회 iter돌 때의 모든 rmse의 평균 (50회의 평균)) 
print("total cv mean : ",round(np.mean(tmp),4)) # mean of cv's mean of 1 iter (각 cv rmse 평균의 평균) # we need var of this value 

#model retrain with all train data  
SVR.fit(x_train,y_train) # > model train
y_pic_test_pred = SVR.predict(x_test) # > model predidction

# 
y_pic_test_pred_df = pd.DataFrame(columns=['y_pic_test_idx','y_pic_test_pred'])
for j in range(len(x_test_idx)):
        y_pic_test_pred_df.loc[x_test_idx[j],'y_pic_test_idx'] = x_test_idx[j]
        y_pic_test_pred_df.loc[x_test_idx[j],'y_pic_test_pred'] = y_pic_test_pred[j]
#y_pic_test_pred_df = pd.DataFrame(y_pic_test_pred,columns=['y_pic_test_pred'])
y_pic_pred_result = pd.concat([y_pic_pred_result, y_pic_test_pred_df], axis=1, ignore_index=False)

#save csv 
#y_pic_pred_result.to_csv(result_wdir+'5-alpha-reductase_dltnan_y_pic_pred_result+with_significant_feature.csv',index=True)


### D.3.3 SVR model results

In [ ]:
#RESULT
print("train data size : {} ({}%)\n".format(len(x_train),round((len(x_train)/len(x)),2)), "test data size : {} ({}%)\n".format(len(x_test),round((len(x_test)/len(x)),2)))
print("each CV RMSE average : \n{} \nVARIANCE of RMSE of every folds : {}\n".format(tmp,round(np.var(cv_rmse),4)))

print("total 50 folds RMSE mean : ", round(np.mean(cv_rmse),4)) # mean of 50 cv rmse (1회 iter돌 때의 모든 rmse의 평균 (50회의 평균)) 
print("total 10 CVs RMSE mean : ",round(np.mean(tmp),4)) # mean of cv's mean of 1 iter (각 cv rmse 평균의 평균) # we need var of this value 

# test rseult 
print("\ntest RMSE : {}".format(round(np.sqrt(mean_squared_error(y_test,y_pic_test_pred)),4)) )

In [ ]:
'''
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect as _GetMorganFingerprintAsBitVect, GetMorganFingerprint as _GetMorganFingerprint
from bioalerts import LoadMolecules, Alerts, FPCalculator 
from rdkit import Chem 

# >>> _GetMorganFingerprint #Returns a Morgan fingerprint for a molecule
# >>> _GetMorganFingerprintAsBitVect  #Returns a Morgan fingerprint for a molecule as a bit vector

molecules = LoadMolecules.LoadMolecules("./tutorial/datasets/5AR.smi",name_field=None)
molecules.ReadMolecules() 
stride = int(len(molecules.mols)*0.9)
training = molecules.mols[0:stride]
test = molecules.mols[stride:len(molecules.mols)]
print (len(molecules.mols), len(test), len(training))

radii = [2,3,4,5,6]


def extract_substructure_information(radii,mols):
    substructure_dictionary = {}
    for i,m in enumerate(mols):
        info = {}
        fp = _GetMorganFingerprint(m,max(radii),bitInfo=info)
        for k,v in info.items():
            if v[0][1] in radii:
                if k in substructure_dictionary.keys():
                    substructure_dictionary[k].append(i)
                else : 
                    substructure_dictionary.update({k:[i]})
    return substructure_dictionary

'''